# FIAP
## Engenharia de Software
### Data Science and Statistical Computing

SQLite LAB - ETL - Extraction Transformation and Load

Install:

pip3 install sqlite3

or

conda install sqlite3

In [ ]:
!wget https://github.com/danielscarvalho/data/blob/master/Clients.csv?raw=true -O Clients.csv

In [1]:
!ls -l *.csv

-rw-rw-r-- 1 fito fito 676762 mai  9 10:51 bitcoins_trades.csv
-rw-rw-r-- 1 fito fito 185213 jun  6 10:32 Cities1.csv
-rw-rw-r-- 1 fito fito  74239 set 26 11:15 Clients.csv
-rw-rw-r-- 1 fito fito  94283 ago 26 09:50 diabetes.csv
-rw-rw-r-- 1 fito fito 554679 abr  8 08:25 obitos-2023.csv
-rw-rw-r-- 1 fito fito  60302 mar 18  2024 titanic.csv


In [2]:
!head Clients.csv

Seq,Value,Cost,Type,Category,Reference,Sub,Dec,Ticket
1,0.993190858335105,704.151859720177,A,Gold,399.122282809757,-70,Gamma,1432
2,0.555051310059166,776.584086430537,B,Silver,828.930698378043,74,Gamma,1433
3,0.0890770883227318,909.221706181418,B,Silver,987.120310195753,-55,Gamma,1434
4,0.109345778842911,57.2442840655498,B,Silver,980.133134243492,87,Gamma,1435
5,0.290697424748017,898.92160632871,B,Silver,190.085343055745,-39,Gamma,1436
6,0.591107732196357,126.598489368848,B,Silver,651.264660866354,83,Gamma,1437
7,0.985322702594319,588.142563005177,C,Silver,252.743863713731,-43,Gamma,1438
8,0.331527780422579,832.966968426264,C,Silver,142.750118211529,50,Gamma,1439
9,0.311497095693109,193.32694327513,C,Silver,480.384255226109,-62,Gamma,1440


Tipo de dados do SQLite e outros SGBDs:

https://www.tutorialspoint.com/sqlite/sqlite_data_types.htm



In [16]:
CT_CLIENTS_SQL = """
create table clients (
seq integer not null primary key,
value float,
cost float,
type text,
category text,
reference integer,
sub integer,
dec text,
ticket interger
)
"""

In [4]:
import sqlite3

con = sqlite3.connect("fiap2.db")

In [5]:
cursor = con.cursor()

In [7]:
result_set = cursor.execute("select date(), time(), sqlite_version()")

In [8]:
for record in result_set:
    print(record)

('2024-09-27', '01:15:04', '3.41.2')


In [17]:
cursor.execute("drop table clients")

In [18]:
result_set = cursor.execute(CT_CLIENTS_SQL)

In [19]:
for record in result_set:
    print(record)

In [20]:
file = open("Clients.csv","r")

clients_head = file.readline()

print(clients_head.split(","))

['Seq', 'Value', 'Cost', 'Type', 'Category', 'Reference', 'Sub', 'Dec', 'Ticket\n']


Data sample, first and second lines of the clients.csv:

```csv
Seq,Value,Cost,Type,Category,Reference,Sub,Dec,Ticket
1,0.993190858335105,704.151859720177,A,Gold,399.122282809757,-70,Gamma,1432
```

In [21]:
for line in file:
    
    try:

        columns = line.split(",")

        seq = int(columns[0])
        value = float(columns[1])
        cost = float(columns[2])
        type = columns[3]
        category = columns[4]
        reference = float(columns[5])
        sub = int(columns[6])
        dec = columns[7]
        ticket = int(columns[8])

        INSERT_SQL = f"insert into clients values({seq},{value},{cost},'{type}','{category}',{reference},{sub},'{dec}',{ticket})"
        
        #print(INSERT_SQL)        
    
        cursor.execute(INSERT_SQL)

    except Exception as err:
        print("Ops! - ", err, " - ", INSERT_SQL)

In [22]:
result_set = cursor.execute("select count(1) qtd from clients")

for record in result_set:
    print(record)

(1000,)


In [23]:
result_set = cursor.execute("select * from clients where seq == (select max(seq) from clients)")

for record in result_set:
    print(record)

(1000, 0.0432759565581071, 920.506722271134, 'B', 'Prata', 687.753336916526, 49, 'Alpha', 1600)


In [24]:
result_set = cursor.execute("select * from clients where reference == (select max(reference) from clients)")

for record in result_set:
    print(record)

(368, 0.154303628511627, 776.115530755998, 'E', 'Prata', 996.557952125095, -6, 'Beta', 1799)


In [25]:
result_set = cursor.execute("select category, count(1) qtd, sum(reference) from clients group by category")

for record in result_set:
    print(record)

('Gold', 16, 5882.483505698788)
('Prata', 776, 391068.11619044293)
('Silver', 208, 108943.19602267281)


In [26]:
import pandas as pd

In [27]:
pd.read_sql_query("select sqlite_version(), date(), time()", con)

,sqlite_version(),date(),time()
0,3.41.2,2024-09-27,01:28:44


In [28]:
pd.read_sql_query("select * from clients where type = 'A' and dec == 'Gamma'", con)

,seq,value,cost,type,category,reference,sub,dec,ticket
0,1,0.993191,704.15186,A,Gold,399.122283,-70,Gamma,1432


To learn more about Pandas with DB: 

https://pandas.pydata.org/pandas-docs/version/0.15.0/generated/pandas.read_sql.html <br>
https://pandas.pydata.org/docs/reference/api/pandas.read_sql.html <br>
https://pandas.pydata.org/docs/reference/api/pandas.read_sql_query.html <br>

In [107]:
file.close()
con.close()

In [ ]:
# cursor.execute("delete from clients")
print(_)